# OpenAI Gym
[Gym](https://www.gymlibrary.dev/) is an open-source Python library, made for easy development and testing of reinforcement learning algorithms. Today we will use it to create and run an instance of the Atari Breakout game. The Gym library provies us access to the game state, game rewards, and available actions, which if you remember are nesseicary for our RL framework. 

<img src="Test.gif" width="200" align="center">

### The RL Framework for Breakout:
- **Action:** Move the paddle left and right 
- **State:** The 210x160 RGB image frame 
- **Reward:** Amount the game score increases 


### Packages

We import the following libraries that are required for this demo.

In [1]:
import gym 
import cv2
import time 
import imageio

In [2]:
from ale_py import ALEInterface
ale = ALEInterface()

from ale_py.roms import Breakout
ale.loadROM(Breakout)

A.L.E: Arcade Learning Environment (version 0.8.0+919230b)
[Powered by Stella]
Game console created:
  ROM file:  /Users/justinvalentine/opt/anaconda3/envs/Test/lib/python3.9/site-packages/ale_py/roms/breakout.bin
  Cart Name: Breakout - Breakaway IV (1978) (Atari)
  Cart MD5:  f34f08e5eb96e500e851a80be3277a56
  Display Format:  AUTO-DETECT ==> NTSC
  ROM Size:        2048
  Bankswitch Type: AUTO-DETECT ==> 2K

Running ROM file...
Random seed is 1667373141


In [ ]:
env = gym.make("ALE/Breakout-v5")
env.reset()
image_lst = []
step_num, total_reward = 0, 0

while step_num < 100:
    action = env.env.action_space.sample()
    state, reward, terminated, truncated, info = env.step(action)

    img = cv2.resize(state, (160, 210), 
                     interpolation=cv2.INTER_CUBIC)
    
    
    frame = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    image_lst.append(frame)
        
    time.sleep(0.01)

    step_num += 1

env.close()

imageio.mimsave('Test.gif', image_lst, fps=30)

<img src="Test.gif" width="300" align="center">